In [1]:
import psycopg2
from dotenv import load_dotenv
load_dotenv()
import os

conn = psycopg2.connect(
    host=os.getenv('DB_HOST'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_NAME'),
    password=os.getenv('DB_PASSWORD')
)

cursor = conn.cursor()

In [2]:
def inserir_comando(sql, conn, **args):
    try:
        cursor.execute(sql, tuple(args.values()))
        conn.commit()
        print('Comando criado')
    except Exception as erro:
        conn.rollback()
        print(erro)

In [3]:
sql_criacao_tabela_usuario = rf''' 
    CREATE TABLE IF NOT EXISTS usuario (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        login VARCHAR(45) NOT NULL,
        senha VARCHAR(45) NOT NULL,
        cargo VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [4]:
inserir_comando(sql=sql_criacao_tabela_usuario, conn=conn)

Comando criado


In [5]:
sql_criacao_tabela_permissao = '''
    CREATE TABLE IF NOT EXISTS permissao (
        id SERIAL PRIMARY KEY,
        nome VARCHAR(45) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [6]:
inserir_comando(sql=sql_criacao_tabela_permissao, conn=conn)

Comando criado


In [7]:
sql_criacao_tabela_usuario_permissao = '''
    CREATE TABLE IF NOT EXISTS usuario_permissao (
        usuario_id INT NOT NULL,
        permissao_id INT NOT NULL,
        
        PRIMARY KEY (usuario_id, permissao_id),
        
        FOREIGN KEY (usuario_id) REFERENCES usuario(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (permissao_id) REFERENCES permissao(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [8]:
inserir_comando(sql=sql_criacao_tabela_usuario_permissao, conn=conn)

Comando criado


In [9]:
sql_usuario = """
INSERT INTO usuario (nome, login, senha, cargo)
VALUES (%s, %s, %s, %s)
RETURNING id;
"""

cursor.execute(
    sql_usuario,
    ("Maria Silva", "maria", "hash_senha", "Analista")
)

conn.commit()

usuario_id = cursor.fetchone()[0]

In [10]:
sql_alterando_tabela = '''
    ALTER TABLE permissao
    ADD CONSTRAINT permissao_nome_unique UNIQUE (nome);
'''

In [11]:
inserir_comando(sql=sql_alterando_tabela, conn=conn)

ERRO:  relação "permissao_nome_unique" já existe



In [12]:
try:
    sql_permissao = """
    INSERT INTO permissao (nome)
    VALUES (%s)
    ON CONFLICT (nome) DO NOTHING
    RETURNING id;
    """

    cursor.execute(sql_permissao, ("ADMIN",))

    
    result = cursor.fetchone()
    
    conn.commit()

    if result:
        permissao_id = result[0]
    else:
        cursor.execute(
            "SELECT id FROM permissao WHERE nome = %s;",
            ("ADMIN",)
        )
        permissao_id = cursor.fetchone()[0]
except Exception as erro:
    conn.rollback()

In [13]:
#vinculos bitch
sql_usuario_permissao = """
INSERT INTO usuario_permissao (usuario_id, permissao_id)
VALUES (%s, %s)
ON CONFLICT DO NOTHING;
"""


inserir_comando(sql=sql_usuario_permissao, conn=conn, usuario_id=4, permissao_id=2)

ERRO:  inserção ou atualização em tabela "usuario_permissao" viola restrição de chave estrangeira "usuario_permissao_usuario_id_fkey"
DETAIL:  Chave (usuario_id)=(4) não está presente na tabela "usuario".



### Tabéla de PRESTADOR/ ENDEREÇO/ REGIME_ATENDIMENTO

In [ ]:
# tabela do prestador bitch 
sql_criacao_tabela_prestador = rf''' 
    CREATE TABLE IF NOT EXISTS prestador (
        id SERIAL PRIMARY KEY,
        cnpj VARCHAR(14),
        nome_fantasia VARCHAR(255) NOT NULL,  
        razao_social VARCHAR(255) NOT NULL,
        telefone VARCHAR(11) NOT NULL,
        horario_abertura TIME NOT NULL, 
        atende_feriado TIMESTAMP NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [21]:
inserir_comando(sql=sql_criacao_tabela_prestador, conn=conn)

Comando criado


In [ ]:
# tabela do regime de atendimento oras bitch 
sql_criacao_tabela_regime_atendimento = '''
    CREATE TABLE IF NOT EXISTS regime_atendimento (
        id SERIAL PRIMARY KEY,
        regime_atendimento VARCHAR(20) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [24]:
inserir_comando(sql=sql_criacao_tabela_regime_atendimento, conn=conn)

Comando criado


In [ ]:
# tabela do regime de atendimento do prestador bitch (o que seria o regime atendimento hein)
sql_criacao_tabela_prestador_regime_atendimento = '''
    CREATE TABLE IF NOT EXISTS prestador_regime_atendimento (
        prestador_id INT NOT NULL,
        regime_atendimento_id INT NOT NULL,
        
        PRIMARY KEY (prestador_id, regime_atendimento_id),
        
        FOREIGN KEY (prestador_id) REFERENCES prestador(id) ON DELETE CASCADE ON UPDATE CASCADE,
        FOREIGN KEY (regime_atendimento_id) REFERENCES regime_atendimento(id) ON DELETE CASCADE ON UPDATE CASCADE
    )
'''

In [26]:
inserir_comando(sql=sql_criacao_tabela_prestador_regime_atendimento, conn=conn)

Comando criado


In [ ]:
# tabela do endereco do preatador bitch 
sql_criacao_tabela_endereco = rf''' 
    CREATE TABLE IF NOT EXISTS endereco_prestador (
        id SERIAL PRIMARY KEY,
        uf VARCHAR(2),
        cidade VARCHAR(100) NOT NULL,  
        bairro VARCHAR(100) NOT NULL,
        logradouro VARCHAR(255) NOT NULL,
        created_at TIMESTAMP NOT NULL DEFAULT NOW(),
        prestador_id INT,
        updated_at TIMESTAMP,
        deleted_at TIMESTAMP
    )
'''

In [32]:
inserir_comando(sql=sql_criacao_tabela_endereco, conn=conn)

Comando criado
